In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# ---------------------------------------------------------
# 1. データ準備ブロック：カラー画像(CIFAR-10)用
# ---------------------------------------------------------
# 平均と標準偏差はCIFAR-10固有の値を使います
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],  # RGB 3つの平均
        std=[0.2470, 0.2435, 0.2616]    # RGB 3つの標準偏差
    )
])

# データをダウンロード（MNIST = 手書き数字）
# ＜＜ここをCIFAR10に変更することになります＞＞
# データセットを CIFAR10 に変更
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# クラス名（数字ではなく名前がついています）
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# ---------------------------------------------------------
# 2. モデル定義ブロック（AIの脳みそ）RGB対応 & サイズ調整
# ---------------------------------------------------------
class SimpleCNN_CIFAR(nn.Module):
    def __init__(self):
        super().__init__()
        # ★変更点1：入力チャンネルが 1(白黒) → 3(カラー)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1)

        # 特徴を圧縮する層（層の数は変わりませんが、流れるデータ量が変わります）
        # ★変更点2：全結合層の入り口のサイズ（計算式は後述）
        # MNISTのときは 5408 でしたが、今回は 7200 になります
        self.fc1 = nn.Linear(7200, 128) # ＜＜ここがラスボス（サイズ計算）＞＞
        self.fc2 = nn.Linear(128, 10)   # 0~9の10種類なのでここはそのまま

        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2)

    def forward(self, x):
        # [Batch, 3, 32, 32] で入ってくる
        x = self.relu(self.conv1(x))
        # → Convを通って [Batch, 32, 30, 30] になる (32-3+1=30)

        x = self.pool(x)
        # → Poolを通って [Batch, 32, 15, 15] になる (30/2=15)

        x = x.view(x.size(0), -1)
        # → ここで一列に並べると... 32 * 15 * 15 = 7200個！

        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# ---------------------------------------------------------
# 3. 学習実行ブロック（ここはほとんど変更なし！）
# ---------------------------------------------------------
# モデルと最適化ツールの準備
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SimpleCNN_CIFAR().to(device) # <--- Fix applied here
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print("学習を開始します... (画像がカラフルなので少し重いかも？)")

# 1回だけ学習（Epoch 1）して動作確認
model.train()
for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)

    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

    if batch_idx % 100 == 0:
        print(f'Batch {batch_idx}: Loss {loss.item():.4f}')
        # 動作確認のため、最初の400バッチくらいで一旦ストップさせます
        if batch_idx >= 400:
            print("動作確認完了！ループを抜けます。")
            break

print("学習完了！")

def imshow(img):
    # Normalizeしたものを元に戻す計算（見やすくするため）
    img = img / 2 + 0.5
    npimg = img.cpu().numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

print("\n--- 実際のデータを見てみましょう ---")
# データローダーから1バッチ取り出す
dataiter = iter(train_loader)
images, labels = next(dataiter)

# 最初の4枚を表示
imshow(torchvision.utils.make_grid(images[:4]))
# ラベルを表示
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))